In [1]:
%cd ..
%load_ext autoreload
%autoreload 2

/home/dongmin/userdata/open-score-string-quartets


/home/dongmin/.local/share/virtualenvs/open-score-string-quartets-wd2Cnojv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import os
import sys
import subprocess
import warnings
from typing import Union, Any, Optional
import shutil
from pathlib import Path
from collections import Counter, defaultdict
from operator import itemgetter
from itertools import groupby
from tempfile import NamedTemporaryFile, TemporaryDirectory

import math
import random

import json
import csv
import strictyaml as syaml

from tqdm import tqdm
import matplotlib.pyplot as plt

import cv2
import numpy as np
import partitura as pt

import xml.etree.ElementTree as ET
from lmx_utils import delinearize_lmx
from lmx_utils.Linearizer import Linearizer, SQLinearizer
from lmx_utils.Delinearizer import Delinearizer
from lmx_utils.symbolic.MxlFile import MxlFile
from lmx_utils.symbolic.part_to_score import part_to_score

dformat = lambda d: json.dumps(d, indent=2)
dprint = lambda d: print(dformat(d))

In [3]:
PathLike = Union[Path, str]

In [4]:
root_dir = Path(os.getcwd())
data_dir = root_dir / 'data'
score_dir = root_dir / 'scores'

In [ ]:
data_dir, score_dir

(PosixPath('/home/dongmin/userdata/open-score-string-quartets/data'),
 PosixPath('/home/dongmin/userdata/open-score-string-quartets/scores'))

## Load metadata

In [6]:
with open(data_dir / 'scores.yaml') as f:
  score_metadata = syaml.load(f.read())

score_metadata = score_metadata.data

mscore_data = [ dict( mscore_id=mscore_id, **obj ) for mscore_id, obj in score_metadata.items() ]

In [7]:
error_cases = []

for item in mscore_data:
  sub_dir, mscore_id = itemgetter('path', 'mscore_id')(item)
  xml_path = score_dir / sub_dir / f'sq{mscore_id}.musicxml'
  
  with open(xml_path) as f:
    xml = f.read()
  
  try:
    xml_tree = ET.ElementTree(ET.fromstring(xml))
  
  except:
    error_cases.append(item)

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7706ab57baf0>>
Traceback (most recent call last):
  File "/home/dongmin/.local/share/virtualenvs/open-score-string-quartets-wd2Cnojv/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


In [9]:
for item in error_cases:
  mscore_dir = score_dir / item['path'] 
  mscore_path = mscore_dir / f"sq{item['mscore_id']}.mscx"
  
  out_path = mscore_dir / f"sq{item['mscore_id']}.musicxml"
  
  cmd = ['musescore3', "-o", out_path, mscore_path, '-f']
    
  try:
    # convert MuseScore file to MusicXML
    ps = subprocess.run(
      cmd, 
      stdout=subprocess.DEVNULL, stderr=subprocess.PIPE
    )
    
    if ps.returncode != 0:
      raise Exception(
        "Command {} failed with code {}. MuseScore " "error messages:\n {}"
        .format(cmd, ps.returncode, ps.stderr.decode("UTF-8"))
      )
  
  except Exception as e:
    raise Exception(
      'Executing "{}" returned  {}.'.format(" ".join(cmd), e)
    )

## Proto: Parse MusicXML to SQLMXE
* SQLMXE: combined measure-wise String Quartet LMX by page

In [7]:
# sub_dir, mscore_id = itemgetter('path', 'mscore_id')(mscore_data[0])
# mscore_id = '7362818'
mscore_id = '7300376'
sub_dir = score_dir / score_metadata[mscore_id]['path']
mscore_path = score_dir / sub_dir / f'sq{mscore_id}.musicxml'

mscore_path.exists()

True

In [8]:
with open(mscore_path, 'r') as f:
  mxml = f.read()

xml_tree = ET.ElementTree(ET.fromstring(mxml))

In [9]:
part_list = xml_tree.getroot().find('part-list')
for part in part_list.findall('score-part'):
  print(part.find("score-instrument/instrument-name").text)

Violin
Violin
Viola
Violoncello


In [10]:
mxl = MxlFile(xml_tree)

mxl.get_string_quartet_parts()

[<Element 'part' at 0x74d39b520630>,
 <Element 'part' at 0x74d39a5ffd80>,
 <Element 'part' at 0x74d399ce5990>,
 <Element 'part' at 0x74d3994cce50>]

In [11]:
mxl = MxlFile(xml_tree)

try:
  sq_parts = mxl.get_string_quartet_parts()
except:
  sq_parts = mxl.tree.find("part")

if len(sq_parts) != 4 or any([p.tag != "part" for p in sq_parts]):
  print("No <part> element found.", file=sys.stderr)
  exit()

sq_linearizer = SQLinearizer(
  errout=sys.stderr
)
lmx_parts = sq_linearizer.process_multi_parts(sq_parts)

[ERROR][P:P1 M:X2]: Measure rest does not have expected duration. Divisions: 24 Measure duration: 72 b'<note>\n        <rest />\n        <duration>24</duration>\n        <voice>1</voice>\n        <type>quarter</type>\n        </note>\n      '
[ERROR][P:P2 M:X1]: Measure rest does not have expected duration. Divisions: 24 Measure duration: 72 b'<note>\n        <rest />\n        <duration>24</duration>\n        <voice>1</voice>\n        <type>quarter</type>\n        </note>\n      '
[ERROR][P:P2 M:X2]: Measure rest does not have expected duration. Divisions: 24 Measure duration: 72 b'<note>\n        <rest />\n        <duration>24</duration>\n        <voice>1</voice>\n        <type>quarter</type>\n        </note>\n      '
[ERROR][P:P2 M:X3]: Measure rest does not have expected duration. Divisions: 24 Measure duration: 72 b'<note>\n        <rest />\n        <duration>24</duration>\n        <voice>1</voice>\n        <type>quarter</type>\n        </note>\n      '
[ERROR][P:P2 M:X4]: Measure 

In [12]:
print(lmx_parts[0])
print(lmx_parts[1])

['measure', 'key:fifths:-3', 'time', 'beats:4', 'beat-type:4', 'clef:G2', 'E4', 'voice:1', 'half', 'stem:up', 'slur:start', 'F4', 'half', 'slur:stop', 'measure', 'D4', 'voice:1', 'quarter', 'stem:up', 'slur:start', 'B4', 'quarter', 'G4', 'quarter', 'E4', 'quarter', 'slur:stop', 'measure', 'C4', 'voice:1', 'quarter', 'stem:up', 'slur:start', 'E4', 'quarter', 'slur:stop', 'B3', 'quarter', 'slur:start', 'E4', 'quarter', 'slur:stop', 'measure', 'F4', 'voice:1', 'eighth', 'dot', 'stem:up', 'beam:begin', 'slur:start', 'G4', '16th', 'beam:end', 'beam:backward-hook', 'slur:stop', 'A4', 'eighth', 'dot', 'beam:begin', 'slur:start', 'G4', '16th', 'beam:end', 'beam:backward-hook', 'slur:stop', 'F4', 'quarter', 'staccato', 'rest', 'quarter', 'measure', 'E4', 'voice:1', 'half', 'stem:up', 'slur:start', 'F4', 'half', 'slur:stop', 'measure', 'D4', 'voice:1', 'quarter', 'stem:up', 'slur:start', 'B4', 'quarter', 'G4', 'quarter', 'E4', 'quarter', 'slur:stop', 'measure', 'D4', 'voice:1', 'quarter', 'flat'

In [13]:
def split_tokens_by_key_token(token_list, key_token, include_key=False):
  groups = []
  
  cur_group = []
  
  for tok in token_list:
    if tok == key_token:
      if len(cur_group) > 0:
        groups.append(cur_group)
        cur_group = []
      
      if include_key:
        groups.append([tok])
      
      continue
    
    cur_group.append(tok)
  
  else:
    groups.append(cur_group)
  
  return groups

In [14]:
pages_by_parts = [
  split_tokens_by_key_token(part, "<new-page>")
  for part in lmx_parts
]

# check number of pages in each part
all( [ len(pages_by_parts[0]) == len(pages) for pages in pages_by_parts[1:] ] )

True

In [15]:
len(pages_by_parts[0]), pages_by_parts[0][0]

(23,
 ['measure',
  'key:fifths:-3',
  'time',
  'beats:4',
  'beat-type:4',
  'clef:G2',
  'E4',
  'voice:1',
  'half',
  'stem:up',
  'slur:start',
  'F4',
  'half',
  'slur:stop',
  'measure',
  'D4',
  'voice:1',
  'quarter',
  'stem:up',
  'slur:start',
  'B4',
  'quarter',
  'G4',
  'quarter',
  'E4',
  'quarter',
  'slur:stop',
  'measure',
  'C4',
  'voice:1',
  'quarter',
  'stem:up',
  'slur:start',
  'E4',
  'quarter',
  'slur:stop',
  'B3',
  'quarter',
  'slur:start',
  'E4',
  'quarter',
  'slur:stop',
  'measure',
  'F4',
  'voice:1',
  'eighth',
  'dot',
  'stem:up',
  'beam:begin',
  'slur:start',
  'G4',
  '16th',
  'beam:end',
  'beam:backward-hook',
  'slur:stop',
  'A4',
  'eighth',
  'dot',
  'beam:begin',
  'slur:start',
  'G4',
  '16th',
  'beam:end',
  'beam:backward-hook',
  'slur:stop',
  'F4',
  'quarter',
  'staccato',
  'rest',
  'quarter',
  'measure',
  'E4',
  'voice:1',
  'half',
  'stem:up',
  'slur:start',
  'F4',
  'half',
  'slur:stop',
  'measure'

In [16]:
systems_by_parts = [
  [
    split_tokens_by_key_token(page, "<new-system>", include_key=False)
    for page in part
  ]
  for part in pages_by_parts
]

# check number of systems in each page
all([
  all( [len(pages[0]) == len(part) for part in pages[1:]] )
  for pages in zip(*systems_by_parts)
])

True

In [17]:
measures_by_parts = [
  [
    [
      split_tokens_by_key_token(system, "measure", include_key=False)
      for system in pages
    ]
    for pages in part
  ]
  for part in systems_by_parts
]

# check number of measures in each system
all([
  all([
    all([
      len(systems[0]) == len(measure)
      for measure in systems[1:]
    ])
    for systems in zip(*pages)
  ])
  for pages in zip(*measures_by_parts)
])

False

In [20]:
for p_i, pages in enumerate(zip(*measures_by_parts)):
  for s_i, system in enumerate(zip(*pages)):
    for part_idx, measures in enumerate(system[1:]):
      if len(system[0]) != len(measures):
        print(part_idx+1, p_i+1, s_i+1, len(system[0]), len(measures))

2 9 1 6 5


In [19]:
measures_by_parts[3][21][0]

[['E3', 'voice:1', 'quarter', 'stem:down', 'staccato', 'rest', 'quarter'],
 ['rest',
  'voice:1',
  'eighth',
  'B2',
  'eighth',
  'stem:up',
  'beam:begin',
  'slur:start',
  'E3',
  'eighth',
  'beam:end',
  'slur:stop',
  'rest',
  'eighth'],
 ['rest',
  'voice:1',
  'eighth',
  'G2',
  'eighth',
  'stem:up',
  'beam:begin',
  'slur:start',
  'C3',
  'eighth',
  'beam:end',
  'slur:stop',
  'rest',
  'eighth'],
 ['rest',
  'voice:1',
  'eighth',
  'E2',
  'eighth',
  'stem:up',
  'beam:begin',
  'slur:start',
  'A2',
  'eighth',
  'beam:end',
  'slur:stop',
  'A3',
  '16th',
  'stem:down',
  'beam:begin',
  'beam:begin',
  'slur:start',
  'B3',
  '16th',
  'beam:end',
  'beam:end',
  'slur:stop'],
 ['C4',
  'voice:1',
  '16th',
  'stem:down',
  'beam:begin',
  'beam:begin',
  'staccato',
  'B3',
  '16th',
  'staccato',
  'A3',
  '16th',
  'staccato',
  'G3',
  '16th',
  'beam:end',
  'beam:end',
  'staccato',
  'F3',
  '16th',
  'beam:begin',
  'beam:begin',
  'staccato',
  'E3',
 

In [25]:
lmx_score = []

for pages in zip(*measures_by_parts):
  lmx_page = []
  
  for systems in zip(*pages):
    lmx_system = []
    
    for measure in zip(*systems):
      lmx_measure = ['measure']
      
      for p_i, part in enumerate(measure):
        lmx_measure += [f'part:{p_i+1}'] + part
      
      lmx_measure_str = ' '.join(lmx_measure)
      lmx_system.append(lmx_measure_str)
    
    lmx_page.append(lmx_system)
  
  lmx_score.append(lmx_page)

In [28]:
dprint(lmx_score[0])

[
  [
    "measure part:1 key:fifths:3 time beats:4 beat-type:4 clef:G2 A4 voice:1 half stem:up tied:start A4 eighth beam:begin tied:stop B4 16th beam:begin slur:start B4 16th sharp beam:end beam:end C5 16th stem:down beam:begin beam:begin D5 16th D5 16th sharp F5 16th beam:end beam:end part:2 key:fifths:3 time beats:4 beat-type:4 clef:G2 A4 voice:1 half stem:up tied:start A4 eighth beam:begin tied:stop B4 16th beam:begin slur:start B4 16th sharp beam:end beam:end C5 16th stem:down beam:begin beam:begin D5 16th D5 16th sharp F5 16th beam:end beam:end part:3 key:fifths:3 time beats:4 beat-type:4 clef:C3 A3 voice:1 half stem:up tied:start A3 eighth beam:begin tied:stop B3 16th beam:begin slur:start B3 16th sharp beam:end beam:end C4 16th stem:down beam:begin beam:begin D4 16th D4 16th sharp F4 16th beam:end beam:end part:4 key:fifths:3 time beats:4 beat-type:4 clef:F4 A3 voice:1 half stem:down tied:start A3 eighth beam:begin tied:stop B3 16th beam:begin slur:start B3 16th sharp beam:end 

In [29]:
dprint(
  '\n\n'.join([ '\n'.join(system)  for system in lmx_score[0] ])
)

"measure part:1 key:fifths:3 time beats:4 beat-type:4 clef:G2 A4 voice:1 half stem:up tied:start A4 eighth beam:begin tied:stop B4 16th beam:begin slur:start B4 16th sharp beam:end beam:end C5 16th stem:down beam:begin beam:begin D5 16th D5 16th sharp F5 16th beam:end beam:end part:2 key:fifths:3 time beats:4 beat-type:4 clef:G2 A4 voice:1 half stem:up tied:start A4 eighth beam:begin tied:stop B4 16th beam:begin slur:start B4 16th sharp beam:end beam:end C5 16th stem:down beam:begin beam:begin D5 16th D5 16th sharp F5 16th beam:end beam:end part:3 key:fifths:3 time beats:4 beat-type:4 clef:C3 A3 voice:1 half stem:up tied:start A3 eighth beam:begin tied:stop B3 16th beam:begin slur:start B3 16th sharp beam:end beam:end C4 16th stem:down beam:begin beam:begin D4 16th D4 16th sharp F4 16th beam:end beam:end part:4 key:fifths:3 time beats:4 beat-type:4 clef:F4 A3 voice:1 half stem:down tied:start A3 eighth beam:begin tied:stop B3 16th beam:begin slur:start B3 16th sharp beam:end beam:end C

## Convert openScore SQ Corpus to SQLMXE

In [33]:
from lmx_utils import score_to_lmxe

for item in tqdm(mscore_data, desc='score'):
  sub_dir, mscore_id = itemgetter('path', 'mscore_id')(item)
  xml_path = score_dir / sub_dir / f'sq{mscore_id}.musicxml'
  
  lmxe_dir = score_dir / sub_dir / 'lmxe'
  lmxe_dir.mkdir(exist_ok=True)
  
  with open(xml_path, 'r') as f:
    xml_str = f.read()
  
  lmx_score = score_to_lmxe(xml_str)
  lmx_score
  
  for p_i, page in enumerate(tqdm(lmx_score, desc='page', leave=False)):
    lmxe_str = '\n\n'.join([ '\n'.join(system)  for system in lmx_score[0] ])
    lmxe_path = lmxe_dir / f'sq{mscore_id}:{str(p_i).zfill(4)}.lmxe'
    
    with open(lmxe_path, 'w') as f:
      f.write(lmxe_str)

score:   2%|▏         | 2/122 [00:01<01:49,  1.10it/s][ERROR][P:P2 M:X3]: Measure rest does not have expected duration. Divisions: 24 Measure duration: 48 b'<note>\n        <rest />\n        <duration>12</duration>\n        <voice>1</voice>\n        <type>eighth</type>\n        </note>\n      '
[ERROR][P:P3 M:X3]: Measure rest does not have expected duration. Divisions: 24 Measure duration: 48 b'<note>\n        <rest />\n        <duration>12</duration>\n        <voice>1</voice>\n        <type>eighth</type>\n        </note>\n      '
[ERROR][P:P4 M:X3]: Measure rest does not have expected duration. Divisions: 24 Measure duration: 48 b'<note>\n        <rest />\n        <duration>12</duration>\n        <voice>1</voice>\n        <type>eighth</type>\n        </note>\n      '
score:   2%|▏         | 3/122 [00:03<02:12,  1.11s/it]


UnboundLocalError: local variable 'note_type' referenced before assignment

## Convert system-wise LMX to Music XML

In [19]:
lmx_split = total_lmx.split(' ')

lmx_split = [
  [ list(g) for _, g in groupby(lmx_split, key=lambda x: x != 'measure') ]
]
lmx_split = [ g for g in lmx_split[0] if g[0] != 'measure' ]

lmx_split = [  
  [ list(sg) for _, sg in groupby(g, key=lambda x: 'part:' not in x)]
  for g in lmx_split
]
lmx_split = [
  [ sg for sg in g if 'part:' not in sg[0] ]
  for g in lmx_split
]

lmx_by_parts = [ [] for _ in range(len(lmx_split[0])) ]

for m in lmx_split:
  for p_i, p in enumerate(m):
    p = ['measure'] + p
    lmx_by_parts[p_i] += p

lmx_by_parts = [ ' '.join(p) for p in lmx_by_parts ]

In [20]:
lmx_by_parts[0]

'measure key:fifths:3 time beats:4 beat-type:4 clef:G2 A4 voice:1 half stem:up tied:start A4 eighth beam:begin tied:stop B4 16th beam:begin slur:start B4 16th sharp beam:end beam:end C5 16th stem:down beam:begin beam:begin D5 16th D5 16th sharp F5 16th beam:end beam:end measure E5 voice:1 eighth stem:down slur:stop rest eighth C6 half slur:start B5 eighth beam:begin A5 eighth beam:end slur:stop measure A5 voice:1 quarter stem:down slur:start G5 eighth dot beam:begin F5 16th beam:end beam:backward-hook slur:stop F5 quarter slur:start E5 eighth dot beam:begin D5 16th beam:end beam:backward-hook slur:stop measure C5 voice:1 eighth dot stem:down beam:begin staccato D5 16th beam:end beam:backward-hook staccato E5 quarter rest half measure B4 voice:1 eighth dot stem:down beam:begin slur:start C5 16th beam:end beam:backward-hook D5 half dot tied:start slur:stop measure D5 voice:1 half dot stem:down tied:stop slur:start C5 quarter slur:stop measure C5 voice:1 quarter stem:down staccato F5 quar

In [21]:
lmx_by_parts[1]

'measure key:fifths:3 time beats:4 beat-type:4 clef:G2 A4 voice:1 half stem:up tied:start A4 eighth beam:begin tied:stop B4 16th beam:begin slur:start B4 16th sharp beam:end beam:end C5 16th stem:down beam:begin beam:begin D5 16th D5 16th sharp F5 16th beam:end beam:end measure E5 voice:1 eighth stem:down slur:stop rest eighth C5 half dot measure D5 voice:1 half dot stem:down B4 quarter measure A4 voice:1 eighth dot stem:up beam:begin staccato B4 16th beam:end beam:backward-hook staccato C5 quarter stem:down rest half measure A4 voice:1 whole backup whole print-object:no rest voice:2 half print-object:no rest half measure G4 voice:1 whole backup whole print-object:no rest voice:2 half print-object:no rest half measure F4 voice:1 half dot stem:up F4 quarter measure F4 voice:1 eighth dot stem:up beam:begin G4 16th beam:end beam:backward-hook A4 half slur:start G4 eighth beam:begin F4 eighth beam:end slur:stop measure F4 voice:1 eighth dot stem:up beam:begin G4 16th beam:end beam:backward

In [22]:
part_xml = Delinearizer( errout=sys.stderr ).process_text( lmx_by_parts[1] )

# ET.tostring(part_xml)

for m_i, measure in enumerate(part_xml):
  if m_i != 4:
    continue
  
  for el in measure:
    if el.tag == 'note' and el.get('print-object') == 'no':
      print(el.tag, type(el.get('print-object')))
  # if child.tag == 'note':
  #   print(child.attrib)
    # print(child.get('print-object'))

note <class 'str'>
note <class 'str'>


In [23]:
from lmx_utils.symbolic.part_to_score import string_quartet_parts_to_score

part_el_list = []

for p in lmx_by_parts:
  part_xml = Delinearizer( errout=sys.stderr ).process_text( p )
  
  # prune print-object="no" elements
  for measure in part_xml:
    for el in reversed(measure):
      if el.tag == 'note' and el.get('print-object') == 'no':
        measure.remove(el)
  
  part_el_list.append(part_xml)

score_etree = string_quartet_parts_to_score(part_el_list)

output_xml = str(
  ET.tostring(
    score_etree.getroot(),
    encoding="utf-8",
    xml_declaration=True
  ), 
  "utf-8"
)

In [92]:
render_musicxml(output_xml, out=Path('./test'))

/home/dongmin/.local/share/virtualenvs/open-score-string-quartets-wd2Cnojv/lib/python3.10/site-packages/partitura/io/importmusicxml.py:181: UserWarning: part-group 1 was not ended
  warnings.warn("part-group {0} was not ended".format(current_group.number))
